In [ ]:
import pandas as pd

melb_df = pd.read_csv('data/melb_data_fe.csv')
melb_df.head()

In [ ]:
display(melb_df.info())

In [ ]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'])
display(melb_df.info())

In [ ]:
display(melb_df['Date'].dt.quarter.value_counts())

In [ ]:
unique_list = []
# пробегаемся по именам столбцов в таблице
for col in melb_df.columns:
    # создаём кортеж (имя столбца, число уникальных значений)
    item = (col, melb_df[col].nunique(),melb_df[col].dtype) 
    # добавляем кортеж в список
    unique_list.append(item) 
# создаём вспомогательную таблицу и сортируем её
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique',  ignore_index=True)
# выводим её на экран
display(unique_counts)

In [ ]:
print(unique_list)

In [ ]:
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
display(melb_df.info())

In [ ]:
melb_df.sort_values(by='Price').head(10)

In [ ]:
melb_df.sort_values(by='Date', ascending=False)

In [ ]:
melb_df.sort_values(by=['Distance', 'Price']).loc[::10, ['Distance', 'Price']]

In [ ]:
melb_df.sort_values(by=['Price', 'Distance']).loc[::10, ['Price', 'Distance']]

In [ ]:
# Предположим, компания McGrath поручила нам восстановить хронологию продаж таунхаусов, 
# у которых площадь участка существенно больше площади здания, чтобы понять, 
# как часто компания справляется с таким сложным видом объектов. 
# Объекты, проданные в один и тот же день, мы бы хотели сортировать по значению коэффициента соотношения площадей. 

# Найдём информацию о таунхаусах (Type), проданных компанией (SellerG) McGrath, 
# у которых коэффициент соотношения площадей здания и участка (AreaRatio) меньше -0.8. 
# Результат отсортируем по дате продажи (Date) в порядке возрастания, 
# а после проведём сортировку по убыванию коэффициента соотношения площадей. 
# Также обновим старые индексы на новые, установив параметр ignore_index на True. 
# Для наглядности результата выберем из таблицы только столбцы SellerG и AreaRatio:

mask1 = melb_df['AreaRatio'] < -0.8
mask2 = melb_df['Type'] == 'townhouse'
mask3 = melb_df['SellerG'] == 'McGrath'
melb_df[mask1 & mask2 & mask3].sort_values(
    by=['Date', 'AreaRatio'],
    ascending=[True, False],
    ignore_index=True
).loc[:, ['Date', 'AreaRatio']]

In [ ]:
# Произведите сортировку столбца AreaRatio по убыванию. При этом индексы полученной таблицы замените на новые. 
# Какое значение площади здания находится в строке 1558? Ответ округлите до целого числа.

display(melb_df.sort_values(
    by='AreaRatio',
    ascending=False,
    ignore_index=True
)['AreaRatio'])
melb_df.loc[1558, 'BuildingArea']


In [ ]:
# Найдите таунхаусы (Type) с количеством жилых комнат (Rooms) больше 2. 
# Отсортируйте полученную таблицу сначала по возрастанию числа комнат, 
# а затем по убыванию средней площади комнат (MeanRoomsSquare). 
# Индексы таблицы замените на новые. Какая цена будет у объекта в строке 18? Ответ запишите в виде целого числа.

mask1 = melb_df['Type'] == 'townhouse'
mask2 = melb_df['Rooms'] > 2
display(melb_df[mask1 & mask2].sort_values(
    by=['Rooms','MeanRoomsSquare'],
    ascending=[True,False],
    ignore_index=True
).loc[18, 'Price'])

МЕТОД GROUPBY()

In [ ]:
melb_df.groupby(by='Type').mean()

In [ ]:
melb_df.groupby('Type')['Price'].mean()

In [ ]:
# Теперь давайте выясним, какие регионы (Regionname) наиболее удалены от центра Мельбурна.
# Для этого найдём минимальное значение расстояния от центра города до объекта в зависимости от его региона. 
# Результат отсортируем по убыванию расстояния:

melb_df.groupby('Regionname')['Distance'].min().sort_values(ascending=False)

In [ ]:
# Давайте построим таблицу для анализа продаж по месяцам. 
# Для этого найдём количество продаж, а также среднее и максимальное значения цен объектов недвижимости (Price), 
# сгруппированных по номеру месяца продажи (MonthSale). 
# Результат отсортируем по количеству продаж в порядке убывания:

melb_df.groupby('MonthSale')['Price'].agg(
    ['count', 'mean', 'max']
).sort_values(by='count', ascending=False)

In [ ]:
melb_df.groupby('MonthSale')['Price'].agg('describe')

In [ ]:
# После базовых математических функций наиболее частым агрегированием является подсчёт числа уникальных значений. 
# Так, например, мы можем вычислить число уникальных риелторских компаний в зависимости от региона, чтобы понять, 
# в каких регионах конкуренция на рынке недвижимости меньше. Это можно сделать, передав в параметр метода agg() строку 'nunique'. 

# Более того, метод agg() поддерживает использование и других функций. 
# Передадим дополнительно встроенную функцию set, чтобы получить множество из агентств недвижимости, которые работают в каждом из регионов:

melb_df.groupby('Regionname')['SellerG'].agg(
    		['nunique', set]
)

In [ ]:
# Сгруппируйте данные по признаку количества комнат и найдите среднюю цену объектов недвижимости в каждой группе. 
# В качестве ответа запишите количество комнат, для которых средняя цена наибольшая.

melb_df.groupby('Rooms')['Price'].mean().sort_values(ascending=False)
    

In [ ]:
# Какой регион имеет наименьшую протяжённость по географической широте (Lattitude)?
# Для ответа на этот вопрос рассчитайте стандартное отклонение широты для каждого региона.
# В качестве ответа запишите название этого региона.

melb_df.groupby('Regionname')['Lattitude'].std().sort_values()

In [ ]:
# Какая риелторская компания (SellerG) имеет наименьшую общую выручку за период с 1 мая по 1 сентября (включительно) 2017 года?
# Для ответа на этот вопрос рассчитайте сумму продаж (Price) каждой компании в заданный период.
# Не забудьте перевести даты в формат datetime

mask1 = melb_df['Date'] <= '2017-09-01'
mask2 = melb_df['Date'] >= '2017-05-01'
melb_df[mask1 & mask2].groupby('SellerG')['Price'].sum().sort_values()

МЕТОД GROUPBY КАК СПОСОБ ПОСТРОЕНИЯ СВОДНЫХ ТАБЛИЦ

In [ ]:
# Например, мы можем построить зависимость медианной цены и площади здания от числа комнат:
melb_df.groupby(['Rooms'])[['Price', 'BuildingArea']].median()

In [ ]:
# Но можно пойти дальше и построить таблицу, в которой мы будем учитывать не только число комнат, но и тип здания (Type). 
# Для этого в параметрах метода groupby() укажем список из нескольких интересующих нас столбцов. 
# Чтобы финальный результат был представлен в виде таблицы, к результату агрегирования применим метод unstack():

melb_df.groupby(['Rooms', 'Type'])['Price'].mean().unstack()

МЕТОД PIVOT_TABLE ДЛЯ ПОСТРОЕНИЯ СВОДНЫХ ТАБЛИЦ

In [ ]:
melb_df.pivot_table(
    values='Price',
    index='Rooms',
    columns='Type',
     fill_value = 0
).round(2)

In [ ]:
# А теперь давайте проанализируем продажи в каждом из регионов в зависимости от того, будний был день или выходной. 
# Для этого построим сводную таблицу, в которой строками будут являться названия регионов (Regionname), 
# а в столбцах будет располагаться наш «признак-мигалка» выходного дня (Weekend), 
# который равен 1, если день был выходным, и 0 — в противном случае. В качестве значений сводной таблицы возьмём количество продаж.

melb_df.pivot_table(
    values='Price',
    index='Regionname',
    columns='Weekend',
    aggfunc='count',
)

In [ ]:
# Разберём ещё один пример. Найдём, как зависит средняя и медианная площадь участка (Landsize) от типа объекта (Type) и его региона (Regionname). 
# Чтобы посмотреть несколько статистических параметров, нужно передать в аргумент aggfunc список из агрегирующих функций. 
# Построим такую сводную таблицу, где пропущенные значения заменим на 0:

melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)

In [ ]:
# Давайте построим таблицу, в которой по индексам будут располагаться признаки метода продажи (Method) и типа объекта (Type), 
# по столбцам — наименование региона (Regionname), а на пересечении строк и столбцов будет стоять медианная цена объекта (Price):

melb_df.pivot_table(
    values='Price',
    index=['Method','Type'],
    columns='Regionname',
    aggfunc='median',
    fill_value=0
)

In [24]:
pivot = melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)
pivot.columns

MultiIndex([('median',     'house'),
            ('median', 'townhouse'),
            ('median',      'unit'),
            (  'mean',     'house'),
            (  'mean', 'townhouse'),
            (  'mean',      'unit')],
           names=[None, 'Type'])

In [ ]:
display(pivot['mean']['unit'])

In [ ]:
mask = pivot['mean']['house'] < pivot['median']['house']
filtered_pivot = pivot[mask]
display(filtered_pivot)

In [ ]:
print(list(filtered_pivot.index))

In [ ]:
# На самом деле мультииндексные таблицы можно создавать и вручную. Давайте посмотрим на синтаксис данной конструкции:

import numpy as np
mser = pd.Series(
    np.random.rand(8),
	index=[['white','white','white','blue','blue','red','red','red'], 
           ['up','down','right','up','down','up','down','left']])
display(mser)

In [ ]:
print(mser.index)

In [ ]:
# Аналогично создаются DataFrame со вложенными признаками (вложенными столбцами) 
# — для этого вложенный список передаётся в параметр columns при инициализации таблицы:

mframe = pd.DataFrame(
    np.random.randn(16).reshape(4,4),
    index=[['white','white','red','red'], ['up','down','up','down']],
    columns=[['pen','pen','paper','paper'],[1,2,1,2]]
)
display(mframe)

In [ ]:
# Составьте сводную таблицу, которая показывает зависимость медианной площади (BuildingArea) здания 
# от типа объекта недвижимости (Type) и количества жилых комнат в доме (Rooms). 
# Для какой комбинации признаков площадь здания наибольшая?

melb_df.pivot_table(
    values='BuildingArea',
    index=['Type'],
    columns='Rooms',
    aggfunc='median',
    fill_value=0
)

In [ ]:
# Составьте сводную таблицу, которая показывает зависимость средней цены объекта недвижимости (Price) от риелторского агентства (SellerG) и типа здания (Type).
# Во вновь созданной таблице найдите агентство, у которого средняя цена для зданий типа unit максимальна. В качестве ответа запишите название этого агентства.


